In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import functions as F
from pyspark.sql import Window

spark = SparkSession.builder.appName("teste_aula").getOrCreate()

import pandas as pd
import statistics

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

import plotly.express as px
import plotly.offline as pyo 
import plotly.graph_objs as go
from plotly.subplots import make_subplots

pyo.init_notebook_mode(connected=True)

In [2]:
#importing data
demand_data = spark.read.format("csv").load('gs://athena-tmp/rafa/demand_data.csv', header = True)
supply_data = spark.read.format("csv").load('gs://athena-tmp/rafa/supply_data.csv', header = True)

In [3]:
#casting data

demand_data = (demand_data.select(F.col('people_saw_0_cars').cast('int').alias('people_saw_0_cars'),
                                  F.col('people_saw_1_cars').cast('int').alias('people_saw_1_cars'),
                                  F.col('coverage_ratio').cast('float').alias('coverage_ratio'),
                                  'date'
                                 )
                          .orderBy('date',)
              )

supply_data = (supply_data.select(F.col('active_drivers').cast('int').alias('active_drivers'),
                                  F.col('online_h').cast('int').alias('online_h'),
                                  F.col('has_booking_h').cast('int').alias('has_booking_h'),
                                  F.col('waiting_for_booking_h').cast('int').alias('waiting_for_booking_h'),
                                  F.col('hours_per_active_driver').cast('float').alias('hours_per_active_driver'),
                                  F.col('rides_per_online_hours').cast('float').alias('rides_per_online_hours'),
                                  F.col('finished_rides').cast('int').alias('finished_rides'),
                                  'date'
                                 )
                         .orderBy('date')
              )

In [4]:
#historical demand data plot
fig = px.line(demand_data.toPandas(),
              x = "date",
              y = ['people_saw_0_cars','people_saw_1_cars']
             )
fig.show()

In [5]:
#joining data sets
complete_table = (demand_data.withColumn('total_demand', F.col('people_saw_0_cars') + F.col('people_saw_1_cars'))
                             .join(supply_data,'date','left')
                             .withColumn('demand_supply_ratio', F.col('total_demand') / F.col('active_drivers'))
                             .withColumn('hour', F.substring(F.col('date'),12,2))
                             .withColumn('real_date', F.substring(F.col('date'),1,10).cast('date'))
                  )

In [6]:
complete_table.select('total_demand','active_drivers','coverage_ratio').toPandas().describe()

,total_demand,active_drivers,coverage_ratio
count,839.0000,839.0000,839.0000
mean,48.2825,61.6484,75.6615
std,36.1908,25.9282,18.3424
min,1.0000,9.0000,0.0000
25%,25.0000,41.0000,66.0000
50%,40.0000,63.0000,80.0000
75%,64.0000,80.5000,89.0000
max,270.0000,129.0000,100.0000


In [7]:
#aggregating data
complete_table_agg = complete_table.groupBy('hour').agg(F.mean(F.col('total_demand')).alias('avg_demand'),
                                                        F.mean(F.col('active_drivers')).alias('avg_supply'),
                                                        F.mean(F.col('demand_supply_ratio')).alias('avg_ds_ratio'),
                                                        F.mean(F.col('coverage_ratio')).alias('avg_coverage_ratio'),
                                                        F.mean(F.col('people_saw_0_cars')).alias('avg_people_saw_0_cars'),
                                                        F.mean(F.col('finished_rides')).alias('avg_finished_rides'),
                                                       )

In [8]:
# coverage ratio behavior analysis
fig = px.box(complete_table.orderBy('hour').toPandas(),
              x = "hour",
              y = 'coverage_ratio'
             )
fig.show()

In [9]:
# demand/supply ratio behavior analysis
fig = px.box(complete_table.filter(F.col('hour').isin(['08','09','18'])).orderBy('hour').toPandas(),
              x = "hour",
              y = 'demand_supply_ratio'
             )
fig.show()

In [10]:
# demand and supply comparison
fig = px.line(complete_table_agg.orderBy('hour').toPandas(),
              x = "hour",
              y = ['avg_demand','avg_supply'],
              title = 'Average Supply and Demand Comparison per hour of the day',
              labels = 'average amount'
             )
fig.show()

In [11]:
# average coverage ratio per hour
fig = px.bar(complete_table_agg.orderBy('hour').toPandas(),
              x = "hour",
              y = 'avg_coverage_ratio',
              title = 'Average coverage ratio per hour of the day'
             )
fig.show()

In [12]:
# validating undersupply hours at 18 hour
complete_table.filter((F.col('hour') == '18') & (F.col('demand_supply_ratio') > 1)).count()

14

In [13]:
complete_table.filter((F.col('hour') == '09')).count()

35

In [14]:
# checking how much undersupplied bolt was at 8, 9 and 18 hour, based on coverage ratio

coverage_ratio_us = (complete_table.filter((F.col('hour').isin(['08', '09', '18'])) & (F.col('people_saw_0_cars') > 0))
                                   .select('hour','date','people_saw_0_cars')
                    )

In [15]:
coverage_ratio_us.show()

+----+-------------+-----------------+
|hour|         date|people_saw_0_cars|
+----+-------------+-----------------+
|  08|2016-11-14 08|               38|
|  09|2016-11-14 09|               43|
|  18|2016-11-14 18|               11|
|  08|2016-11-15 08|               58|
|  09|2016-11-15 09|               41|
|  18|2016-11-15 18|               13|
|  08|2016-11-16 08|               33|
|  09|2016-11-16 09|               24|
|  18|2016-11-16 18|               16|
|  08|2016-11-17 08|               23|
|  09|2016-11-17 09|               33|
|  18|2016-11-17 18|               34|
|  08|2016-11-18 08|               26|
|  09|2016-11-18 09|               25|
|  18|2016-11-18 18|               16|
|  08|2016-11-19 08|               12|
|  09|2016-11-19 09|               16|
|  18|2016-11-19 18|                6|
|  08|2016-11-20 08|                6|
|  09|2016-11-20 09|                7|
+----+-------------+-----------------+
only showing top 20 rows



In [29]:
# checking how much undersupplied bolt was at 8, 9 and 18 hour, based on demand supply ratio
ds_ratio_us = (complete_table.filter((F.col('hour').isin(['08', '09', '18'])) & (F.col('demand_supply_ratio') > 1))
                             .withColumn('undersupply', F.col('total_demand') - F.col('active_drivers'))
                             .select('hour','date','total_demand','active_drivers','undersupply','rides_per_online_hours','people_saw_0_cars')
              )

In [30]:
ds_ratio_us.show()

+----+-------------+------------+--------------+-----------+----------------------+-----------------+
|hour|         date|total_demand|active_drivers|undersupply|rides_per_online_hours|people_saw_0_cars|
+----+-------------+------------+--------------+-----------+----------------------+-----------------+
|  08|2016-11-14 08|          77|            47|         30|                  1.19|               38|
|  09|2016-11-14 09|          85|            55|         30|                  0.95|               43|
|  08|2016-11-15 08|          93|            44|         49|                  1.07|               58|
|  09|2016-11-15 09|          93|            59|         34|                  1.21|               41|
|  08|2016-11-16 08|          86|            55|         31|                   1.0|               33|
|  09|2016-11-16 09|          91|            71|         20|                  1.07|               24|
|  08|2016-11-17 08|          68|            59|          9|                  0.96

In [31]:
# rides per online hour
fig = px.box(ds_ratio_us.filter(F.col('hour').isin(['08','09','18'])).orderBy('hour').toPandas(),
              x = "hour",
              y = 'rides_per_online_hours'
             )
fig.show()

In [32]:
(ds_ratio_us.filter(F.col('hour').isin(['08'
                                        #,'09'
                                        #,'18'
                                       ])
                   )
            .select('undersupply','rides_per_online_hours','total_demand','people_saw_0_cars')
            .toPandas()
            .describe()
)

,undersupply,rides_per_online_hours,total_demand,people_saw_0_cars
count,21.0000,21.0000,21.0000,21.0000
mean,27.1905,1.0957,79.4762,37.5714
std,20.7210,0.2270,17.6086,16.1511
min,1.0000,0.6700,32.0000,17.0000
25%,9.0000,0.9400,71.0000,26.0000
50%,30.0000,1.0000,78.0000,36.0000
75%,45.0000,1.2600,93.0000,50.0000
max,72.0000,1.5800,112.0000,80.0000


In [33]:
(ds_ratio_us.filter(F.col('hour').isin([#'08'
                                        '09'
                                        #,'18'
                                       ])
                   )
            .select('undersupply','rides_per_online_hours','total_demand','people_saw_0_cars')
            .toPandas()
            .describe()
)

,undersupply,rides_per_online_hours,total_demand,people_saw_0_cars
count,21.0000,21.0000,21.0000,21.0000
mean,36.4762,1.1781,100.0952,41.4762
std,25.9029,0.2406,22.4430,16.3543
min,5.0000,0.8600,62.0000,17.0000
25%,20.0000,1.0000,86.0000,32.0000
50%,30.0000,1.1300,97.0000,39.0000
75%,45.0000,1.3700,111.0000,43.0000
max,116.0000,1.6800,163.0000,90.0000


In [34]:
(ds_ratio_us.filter(F.col('hour').isin([#'08'
                                        #'09'
                                        '18'
                                       ])
                   )
            .select('undersupply','rides_per_online_hours','total_demand','people_saw_0_cars')
            .toPandas()
            .describe()
)

,undersupply,rides_per_online_hours,total_demand,people_saw_0_cars
count,14.0000,14.0000,14.0000,14.0000
mean,56.3571,1.1221,140.7143,53.7143
std,40.0338,0.2064,37.6980,29.7410
min,17.0000,0.8100,91.0000,22.0000
25%,31.2500,1.0550,121.2500,31.7500
50%,41.0000,1.1150,126.5000,49.0000
75%,67.2500,1.1725,156.5000,61.0000
max,139.0000,1.7200,231.0000,124.0000


In [35]:
# undersupply volume
fig = px.box(ds_ratio_us.filter(F.col('hour').isin(['08','09','18'])).orderBy('hour').toPandas(),
              x = "hour",
              y = 'people_saw_0_cars'
             )
fig.show()

In [47]:
# drivers earnings

drivers_earnings_table = (complete_table.filter(F.col('hour').isin(['08', '09', '18']))
                                        .withColumn('earnings_per_drive', F.lit(10))
                                        .withColumn('total_earnings', F.col('finished_rides') * F.col('earnings_per_drive'))
                                        .withColumn('ideal_earnings', (F.col('finished_rides') + F.col('people_saw_0_cars')) * F.col('earnings_per_drive'))
                                        .select('hour','total_earnings','ideal_earnings','active_drivers')
                         )

In [48]:
drivers_earnings_table.filter(F.col('hour') == '08').toPandas().describe()

,total_earnings,ideal_earnings,active_drivers
count,35.0000,35.0000,35.0000
mean,146.8571,402.0000,47.2857
std,91.1938,261.8015,13.9970
min,10.0000,20.0000,23.0000
25%,40.0000,140.0000,39.0000
50%,180.0000,450.0000,47.0000
75%,220.0000,605.0000,60.0000
max,310.0000,970.0000,69.0000


In [49]:
drivers_earnings_table.filter(F.col('hour') == '09').toPandas().describe()

,total_earnings,ideal_earnings,active_drivers
count,33.0000,33.0000,35.0000
mean,213.9394,507.8788,57.1143
std,123.6916,311.6564,17.7942
min,10.0000,70.0000,22.0000
25%,80.0000,160.0000,45.5000
50%,260.0000,560.0000,57.0000
75%,300.0000,700.0000,73.5000
max,370.0000,1250.0000,84.0000


In [50]:
drivers_earnings_table.filter(F.col('hour') == '18').toPandas().describe()

,total_earnings,ideal_earnings,active_drivers
count,35.0000,35.0000,35.0000
mean,266.0000,531.1429,85.5429
std,110.8576,372.9593,12.4105
min,60.0000,80.0000,59.0000
25%,175.0000,250.0000,79.0000
50%,290.0000,450.0000,83.0000
75%,345.0000,725.0000,92.0000
max,470.0000,1670.0000,113.0000
